# CS 584 Assignment 3 -- Language Model

#### Name: Aishwarya Bhethanabotla
#### Stevens ID: 20027553

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## In this assignment, you are required to follow the steps below:
1. Review the lecture slides.
2. Implement N-gram language modeling.
3. Implement RNN language modeling.

**Before you start**
- Please read the code very carefully.
- Install these packages (jupyterlab, matplotlib, nltk, numpy, scikit-learn, tensorflow, tensorflow_addons) using the following command.
```console
pip install -r requirements.txt
```
- It's better to train the Tensorflow model with GPU and CUDA. If they are not available on your local machine, please consider Google CoLab. You can check `CoLab.md` in this assignments.
- You are **NOT** allowed to use other packages unless otherwise specified.
- You are **ONLY** allowed to edit the code between `# Start your code here` and `# End` for each block.

## Part A: 1. N-Gram (60 Points)

In [1]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting unin

In [2]:
import sys


def print_line(*args):
    """ Inline print and go to the begining of line
    """
    args1 = [str(arg) for arg in args]
    str_ = ' '.join(args1)
    print('\r' + str_, end='')

In [3]:
import tensorflow as tf


# If you are going to use GPU, make sure the GPU is in the output
tf.config.list_physical_devices('GPU')

[]

In [4]:
from typing import List, Tuple, Union, Dict

import numpy as np

### 1.1 Load Data & Preprocessing

You will not need to implement the data preprocessing.

In [5]:
import os
import pickle


data_path = '/content/a3-data'

train_sentences = open(os.path.join(data_path, 'train.txt')).readlines()
valid_sentences = open(os.path.join(data_path, 'valid.txt')).readlines()
test_sentences = open(os.path.join(data_path, 'input.txt')).readlines()
print('number of train sentences:', len(train_sentences))
print('number of valid sentences:', len(valid_sentences))
print('number of test sentences:', len(test_sentences))

number of train sentences: 42068
number of valid sentences: 3370
number of test sentences: 3165


In [6]:
import re


class Preprocessor:
    def __init__(self, punctuation=True, url=True, number=True):
        self.punctuation = punctuation
        self.url = url
        self.number = number

    def apply(self, sentence: str) -> str:
        """ Apply the preprocessing rules to the sentence
        Args:
            sentence: raw sentence
        Returns:
            sentence: clean sentence
        """
        sentence = sentence.lower()
        sentence = sentence.replace('<unk>', '')
        if self.url:
            sentence = Preprocessor.remove_url(sentence)
        if self.punctuation:
            sentence = Preprocessor.remove_punctuation(sentence)
        if self.number:
            sentence = Preprocessor.remove_number(sentence)
        sentence = re.sub(r'\s+', ' ', sentence)
        return sentence

    @staticmethod
    def remove_punctuation(sentence: str) -> str:
        """ Remove punctuations in sentence with re
        Args:
            sentence: sentence with possible punctuations
        Returns:
            sentence: sentence without punctuations
        """
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        return sentence

    @staticmethod
    def remove_url(sentence: str) -> str:
        """ Remove urls in text with re
        Args:
            sentence: sentence with possible urls
        Returns:
            sentence: sentence without urls
        """
        sentence = re.sub(r'(https|http)?://(\w|\.|/|\?|=|&|%)*\b', ' ', sentence)
        return sentence

    @staticmethod
    def remove_number(sentence: str) -> str:
        """ Remove numbers in sentence with re
        Args:
            sentence: sentence with possible numbers
        Returns:
            sentence: sentence without numbers
        """
        sentence = re.sub(r'\d+', ' ', sentence)
        return sentence

In [7]:
class Tokenizer:
    def __init__(self, sos_token='<s>', eos_token='</s>', pad_token='<pad>', unk_token='<unk>', mask_token='<mask>'):
        # Special tokens.
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.mask_token = mask_token

        self.vocab = { sos_token: 0, eos_token: 1, pad_token: 2, unk_token: 3, mask_token: 4 }
        self.inverse_vocab = { 0: sos_token, 1: eos_token, 2: pad_token, 3: unk_token, 4: mask_token }
        self.token_occurrence = { sos_token: 0, eos_token: 0, pad_token: 0, unk_token: 0, mask_token: 0 }

        self.preprocessor = Preprocessor()

    @property
    def sos_token_id(self):
        """ Create a property method.
            You can use self.sos_token_id or tokenizer.sos_token_id to get the id of the sos_token.
        """
        return self.vocab[self.sos_token]

    @property
    def eos_token_id(self):
        return self.vocab[self.eos_token]

    @property
    def pad_token_id(self):
        return self.vocab[self.pad_token]

    @property
    def unk_token_id(self):
        return self.vocab[self.unk_token]

    @property
    def mask_token_id(self):
        return self.vocab[self.mask_token]

    def __len__(self):
        """ A magic method that enable program to know the number of tokens by calling:
            ```python
            tokenizer = Tokenizer()
            num_tokens = len(tokenizer)
            ```
        """
        return len(self.vocab)

    def fit(self, sentences: List[str]):
        """ Fit the tokenizer using all sentences.
        1. Tokenize the sentence by splitting with spaces.
        2. Record the occurrence of all tokens
        3. Construct the token to index (self.vocab) map and the inversed map (self.inverse_vocab) based on the occurrence. The token with a higher occurrence has the smaller index

        Args:
            sentences: All sentences in the dataset.
        """
        n = len(sentences)
        for i, sentence in enumerate(sentences):
            if i % 100 == 0 or i == n - 1:
                print_line('Fitting Tokenizer:', (i + 1), '/', n)
            tokens = self.preprocessor.apply(sentence.strip()).split()
            if len(tokens) <= 1:
                continue
            for token in tokens:
                if token == '<unk>':
                    continue
                self.token_occurrence[token] = self.token_occurrence.get(token, 0) + 1
        print_line('\n')

        token_occurrence = sorted(self.token_occurrence.items(), key=lambda e: e[1], reverse=True)
        for token, occurrence in token_occurrence[:-5]:
            token_id = len(self.vocab)
            self.vocab[token] = token_id
            self.inverse_vocab[token_id] = token

        print('The number of distinct tokens:', len(self.vocab))

    def encode(self, sentences: List[str]) -> List[List[int]]:
        """ Encode the sentences into token ids
            Note: 1. if a token in a sentence does not exist in the fit encoder, we ignore it.
                  2. If the number of tokens in a sentence is less than two, we ignore this sentence.
                  3. Note that, for every sentence, we will add an sos_token, i.e., the id of <s> at the start of the sentence,
                     and add an eos_token, i.e., the id of </s> at the end of the sentence.
        Args:
            sentences: Raw sentences
        Returns:
            sent_token_ids: A list of id list
        """
        n = len(sentences)
        sent_token_ids = []
        for i, sentence in enumerate(sentences):
            if i % 100 == 0 or i == n - 1:
                print_line('Encoding with Tokenizer:', (i + 1), '/', n)
            token_ids = []
            tokens = self.preprocessor.apply(sentence.strip()).split()
            for token in tokens:
                if token == '<unk>':
                    continue
                if token in self.vocab:
                    token_ids.append(self.vocab[token])
            if len(token_ids) <= 1:
                continue
            token_ids = [self.sos_token_id] + token_ids + [self.eos_token_id]
            sent_token_ids.append(token_ids)
        print_line('\n')
        return sent_token_ids

In [8]:
tokenizer = Tokenizer()
tokenizer.fit(train_sentences[:2])
print()

token_occurrence = sorted(tokenizer.token_occurrence.items(), key=lambda e: e[1], reverse=True)
for token, occurrence in token_occurrence[:10]:
    print(token, ':', occurrence)
print()
sent_token_ids = tokenizer.encode(train_sentences[:2])
print()
for original_sentence, token_ids in zip(train_sentences[:2], sent_token_ids):
    sentence = [tokenizer.inverse_vocab[token] for token in token_ids]
    print(original_sentence, sentence, '\n')

Fitting Tokenizer: 2 / 2
The number of distinct tokens: 44

n : 2
aer : 1
banknote : 1
berlitz : 1
calloway : 1
centrust : 1
cluett : 1
fromstein : 1
gitano : 1
guterman : 1

Encoding with Tokenizer: 2 / 2

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 ['<s>', 'aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro', 'quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack', 'food', 'ssangyong', 'swapo', 'wachter', '</s>'] 

 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 ['<s>', 'pierre', 'n', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov', 'n', '</s>'] 



In [9]:
tokenizer = Tokenizer()
tokenizer.fit(train_sentences)
train_token_ids = tokenizer.encode(train_sentences)
valid_token_ids = tokenizer.encode(valid_sentences)
test_token_ids = tokenizer.encode(test_sentences)

Fitting Tokenizer: 42068 / 42068
The number of distinct tokens: 9614
Encoding with Tokenizer: 42068 / 42068
Encoding with Tokenizer: 3370 / 3370
Encoding with Tokenizer: 3165 / 3165


### 1.2 Calculate unigram and bigram count (10 points)

In [10]:
from typing import List, Dict

def get_unigram_count(train_token_ids: List[List[int]]) -> Dict:
    """ Calculate the occurrence of each token in the dataset.

    Args:
        train_token_ids: each element is a list of token ids
    Return:
        unigram_count: A map from token_id to occurrence
    """
    unigram_count = {}
    for sent in train_token_ids:
        for token_id in sent:
            unigram_count[token_id] = unigram_count.get(token_id, 0) + 1
    return unigram_count


In [13]:
def get_bigram_count(train_token_ids: List[List[int]]) -> Dict[int, Dict]:
    """ Calculate the occurrence of bigrams in the dataset.

    Args:
        train_token_ids: each element is a list of token ids
    Return:
        bigram_count: A map from token_id to next token occurrence. Key: token_id, value: Dict[token_id -> occurrence]
                      For example, {
                          5: {10: 5, 20: 4}
                      } means (5, 10) occurs 5 times and (5, 20) occurs 4 times.
    """
    bigram_count = {}
    for sent in train_token_ids:

        for i in range(len(sent) - 1):
            current_token = sent[i]
            next_token = sent[i + 1]
            if current_token not in bigram_count:
                bigram_count[current_token] = {}
            bigram_count[current_token][next_token] = bigram_count[current_token].get(next_token, 0) + 1
    return bigram_count


In [14]:
unigram_count = get_unigram_count(train_token_ids)
bigram_count = get_bigram_count(train_token_ids)

### 1.3 BiGram (5 points)

In [15]:
class BiGram:
    def __init__(self, unigram_count, bigram_count):
        self.unigram_count = unigram_count
        self.bigram_count = bigram_count

    def calc_prob(self, w1: int, w2: int) -> float:
        """ Calculate the probability of p(w2 | w1) using the BiGram model.

        Args:
            w1, w2: current token and next token
        Note:
            if the calculated probability is 0, return 1e-5.
        """
        # If w1 is not found or w2 never follows w1, return a tiny probability.
        if w1 not in self.bigram_count or w2 not in self.bigram_count[w1]:
            return 1e-5

        # conditional probability
        prob = self.bigram_count[w1][w2] / self.unigram_count[w1]


        if prob == 0:
            return 1e-5

        return prob

### 1.4 Good Turing (15 points)

In [16]:
import numpy as np
import math
from scipy.optimize import curve_fit
from typing import Dict, Tuple, Union

def power_law(x, a, b):
    """Power law function to fit the number of occurrences."""
    return a * np.power(x, b)



class GoodTuring(BiGram):
    def __init__(self, unigram_count, bigram_count, threshold=100, T_cap=200):
        """
        T_cap: Maximum effective number of unseen continuation types to consider
               when distributing reserved probability mass.
        """
        super().__init__(unigram_count, bigram_count)
        self.threshold = threshold
        self.T_cap = T_cap
        self.bigram_Nc = self.calc_Nc()
        self.bi_c_star, self.bi_N = self.smoothing(self.bigram_Nc)
        # V_cont: no.  of distinct tokens that have ever appeared as a continuation
        self.V_cont = len({w2 for w1 in self.bigram_count for w2 in self.bigram_count[w1]})

    def calc_Nc(self) -> Dict[int, Union[float, int]]:
        """Calculate Nc of bigrams.

        Returns:
            bigram_Nc: A map from count to its frequency.
                       For example, {10: 78} means 78 bigrams occur 10 times.
                       For counts above the threshold, frequency is replaced using a power law.
        """
        bigram_Nc = {}
        for w1 in self.bigram_count:
            for w2, count in self.bigram_count[w1].items():
                bigram_Nc[count] = bigram_Nc.get(count, 0) + 1

        self.replace_large_c(bigram_Nc)
        return bigram_Nc

    def replace_large_c(self, Nc):
        """Replace counts for large c with a power law fit."""
        x, y = zip(*sorted(Nc.items(), reverse=True))
        popt, _ = curve_fit(power_law, x, y, bounds=([0, -np.inf], [np.inf, 0]))
        a, b = popt
        max_count = max(Nc.keys())
        for c in range(self.threshold + 1, max_count + 2):
            Nc[c] = power_law(c, a, b)

    def smoothing(self, Nc: Dict[int, Union[float, int]]) -> Tuple[Dict[int, float], float]:
        """Calculate Good-Turing smoothed counts (c_star) and total mass N.

        Args:
            Nc: Map from bigram count to its frequency (after replacement for large counts)
        Returns:
            c_star: Mapping from original bigram count to smoothed count.
                    For counts below the threshold, c* = (c+1)*(Nc[c+1]/Nc[c]).
                    For counts >= threshold, leave the count unchanged.
            N: Total mass computed as sum(c * Nc[c]) over all counts.
        """
        c_star = {}
        N = 0
        for c in sorted(Nc.keys()):
            N += c * Nc[c]
            if c < self.threshold:
                c_star[c] = (c + 1) * (Nc.get(c + 1, 0) / Nc[c])
            else:
                c_star[c] = c
        return c_star, N

    def calc_prob(self, w1, w2) -> float:
        """Calculate the probability p(w2 | w1) using the Good-Turing model.

        For observed bigrams, use the smoothed count.
        For unseen bigrams, reserve probability mass from count-1 events and distribute it
        uniformly over unseen continuation types (capped by T_cap).

        Args:
            w1, w2: current token and next token (token ids)
        """
        if w1 in self.unigram_count:
            C_w1 = self.unigram_count[w1]
        else:
            return 1e-5

        if w1 in self.bigram_count:
            if w2 in self.bigram_count[w1]:
                c = self.bigram_count[w1][w2]
                c_star = self.bi_c_star.get(c, c)
                prob = c_star / C_w1
            else:
                # Unseen bigram: compute reserved mass from count-1 events.
                N1 = sum(1 for count in self.bigram_count[w1].values() if count == 1)
                p_unseen_total = N1 / C_w1
                seen_continuations = len(self.bigram_count[w1])
                T = self.V_cont - seen_continuations
                # Cap T to T_cap.
                T_eff = T if T < self.T_cap else self.T_cap
                if T_eff > 0:
                    prob = p_unseen_total / T_eff
                else:
                    prob = 1e-5
        else:
            prob = 1e-5

        return max(prob, 1e-5)

def perplexity(model, token_ids):
    """Calculate the perplexity score.

    Args:
        model: the language model (e.g., BiGram, GoodTuring, or KneserNey)
        token_ids: list of token id lists (one per sentence)
    Returns:
        perplexity: the perplexity value
    """
    log_probs = 0
    n_words = 0
    n_sentences = len(token_ids)
    for i, tokens in enumerate(token_ids):
        if i % 100 == 0 or i == n_sentences - 1:
            print(f'Calculating perplexity: {i+1} / {n_sentences}')
        for j in range(1, len(tokens)):
            prob = model.calc_prob(tokens[j-1], tokens[j])
            log_probs += math.log(prob)
            n_words += 1
    perp = math.exp(-log_probs / n_words) if n_words > 0 else float('inf')
    print()
    return perp


### 1.5 Kneser-Ney (15 points)

In [17]:
class KneserNey(BiGram):
    def __init__(self, unigram_count, bigram_count, d=0.75):
        super().__init__(unigram_count, bigram_count)
        self.d = d
        self.lambda_ = self.calc_lambda()
        self.p_continuation = self.calc_p_continuation()

    def calc_lambda(self):
        """Calculate λ(w1) for each token w1.

        λ(w1) = (d / count(w1)) * (number of unique bigrams starting with w1)

        Returns:
            lambda_: A dict mapping token_id (w1) to its λ(w1) value.
        """
        lambda_ = {}
        for w1 in self.bigram_count:
            unique_continuations = len(self.bigram_count[w1])
            if self.unigram_count[w1] > 0:
                lambda_[w1] = (self.d * unique_continuations) / self.unigram_count[w1]
            else:
                lambda_[w1] = 0
        return lambda_

    def calc_p_continuation(self):
        """Calculate the continuation probability p_continuation(w).

        p_continuation(w) = (number of unique preceding tokens for w) / V

        Here V is the vocabulary size (i.e. the number of tokens that ever appear as a word).
        This normalization (instead of using the total number of unique bigrams)
        boosts the back‑off probability.

        Returns:
            p_continuation: A dict mapping token_id (w) to p_continuation(w).
        """
        contexts = {}
        for w1 in self.bigram_count:
            for w2 in self.bigram_count[w1]:
                if w2 not in contexts:
                    contexts[w2] = set()
                contexts[w2].add(w1)

        # For each token w, count the number of unique histories (preceding tokens).
        numerator = { w: len(contexts[w]) for w in contexts }

        V = len(self.unigram_count)
        p_continuation = { w: (numerator[w] / V) for w in numerator }
        return p_continuation

    def calc_prob(self, w1, w2) -> float:
        """Calculate the probability p(w2 | w1) using the Kneser‑Ney model.

        p_KN(w2|w1) = max{ c(w1, w2) - d, 0 } / count(w1)
                        + λ(w1) * p_continuation(w2)

        Args:
            w1, w2: current token and next token (token ids)
        Returns:
            p: The Kneser‑Ney probability of w2 given w1.
        """
        c_w1_w2 = self.bigram_count[w1][w2] if (w1 in self.bigram_count and w2 in self.bigram_count[w1]) else 0
        base_prob = self.p_continuation.get(w2, 0)
        p = max(c_w1_w2 - self.d, 0) / self.unigram_count[w1] + self.lambda_.get(w1, 0) * base_prob
        return p if p > 0 else 1e-5


### Show that perplexity is the exponential of the total loss divided by the number of predictions.

### 1.6 Perplexity (10 points)

In [18]:
import math

def perplexity(model, token_ids):
    """ Calculate the perplexity score.

    Args:
        model: the model you want to evaluate (BiGram, GoodTuring, or KneserNey)
        token_ids: a list of validation token_ids, where each element is a list of token ids for a sentence.
    Return:
        perplexity: the perplexity of the model on the texts
    """
    log_probs = 0
    n = len(token_ids)
    n_words = 0
    for i, tokens in enumerate(token_ids):
        if i % 100 == 0 or i == n - 1:
            print_line('Calculating perplexity:', (i + 1), '/', n)
        # Iterate over each bigram in the sentence.
        for j in range(1, len(tokens)):
            # Calculate the probability of the current token given the previous one.
            prob = model.calc_prob(tokens[j-1], tokens[j])

            log_probs += math.log(prob)
            n_words += 1

    #perplexity using the formula: exp(-average log probability)
    perp = math.exp(-log_probs / n_words) if n_words > 0 else float('inf')
    print('\n')
    return perp


If you implement correctly, the perplexity of bigram will be around 320

In [19]:
bigram = BiGram(unigram_count, bigram_count)

# Perplexity
bigram_perplexity = perplexity(bigram, valid_token_ids)
print(f'The perplexity of Bigram is: {bigram_perplexity:.4f}')

Calculating perplexity: 3352 / 3352

The perplexity of Bigram is: 325.8354


If you implement correctly, the perplexity of good turing will be around 130

In [20]:
gt = GoodTuring(unigram_count, bigram_count, threshold=100)

# Perplexity
gt_perplexity = perplexity(gt, valid_token_ids)
print(f'The perplexity of Good Turing is: {gt_perplexity:.4f}')

Calculating perplexity: 3352 / 3352

The perplexity of Good Turing is: 128.6806


If you implement correctly, the perplexity of good turing will be around 60

In [21]:
kn = KneserNey(unigram_count, bigram_count, d=0.75)

# Perplexity
kn_perplexity = perplexity(kn, valid_token_ids)
print(f'The perplexity of Kneser-Ney is: {kn_perplexity:.4f}')

Calculating perplexity: 3352 / 3352

The perplexity of Kneser-Ney is: 62.5943


### 1.7 Predict the next word given a previous word (5 points)

In [22]:
def predict(model: 'BiGram', w1: int, vocab_size: int):
    """Predict the w2 with the highest probability given w1.

    Args:
        model: A BiGram, GoodTuring, or KneserNey model that has the calc_prob function.
        w1: Current word (token id).
        vocab_size: The number of tokens in the vocabulary.

    Returns:
        result: The predicted token id for w2.
    """
    result = None
    highest_prob = 0
    for w2 in range(1, vocab_size):
        prob = model.calc_prob(w1, w2)
        if prob > highest_prob:
            highest_prob = prob
            result = w2
    return result


In [23]:
np.random.seed(12345)

vocab_size = len(tokenizer)
indexes = np.random.choice(len(test_token_ids), 10, replace=False)
for i in indexes:
    token_ids = test_token_ids[i][1:-1]
    print(' '.join([tokenizer.inverse_vocab[token_id] for token_id in token_ids]) + ' ____')
    pred = predict(gt, token_ids[-1], vocab_size)
    print(f'predicted last token: {tokenizer.inverse_vocab[pred]}')
    print('---------------------------------------------')

sharply falling stock prices do reduce consumer wealth damage business ____
predicted last token: </s>
---------------------------------------------
but robert an official of the association said no ____
predicted last token: longer
---------------------------------------------
it also has interests in military electronics and marine ____
predicted last token: s
---------------------------------------------
first chicago since n has reduced its loans to such ____
predicted last token: as
---------------------------------------------
david m jones vice president at g ____
predicted last token: s
---------------------------------------------
the n stock specialist firms on the big board floor ____
predicted last token: traders
---------------------------------------------
at the same time the business was hurt by ____
predicted last token: the
---------------------------------------------
salomon will cover the warrants by buying sufficient shares or ____
predicted last token: n
--------

## Part B: 2. RNN (35 Points)

### 2.1 Split feature and label

In [ ]:
def get_feature_label(token_ids: List[List[int]], window_size: int=-1):
    """ Split features and labels for the training, validation, and test datasets.

    Note:
        If window size is -1, for a sentence with n tokens,
            it selects the tokens rangeing from [0, n - 1) as the feature,
            and selects tokens ranging from [1, n) as the label.
        Otherwise, it divides a sentence with multiple windows and do the previous split.
    """
    x = []
    y = []
    seq_lens = []
    for sent_token_ids in token_ids:
        if window_size == -1:
            x.append(sent_token_ids[:-1])
            y.append(sent_token_ids[1:])
            seq_lens.append(len(sent_token_ids) - 1)
        else:
            if len(sent_token_ids) > window_size:
                sub_sent_size = window_size + 1
                n_window = len(sent_token_ids) // (sub_sent_size)
                for i in range(n_window):
                    start = i * sub_sent_size
                    sub_sent = sent_token_ids[start:(start + sub_sent_size)]
                    x.append(sub_sent[:-1])
                    y.append(sub_sent[1:])
                    seq_lens.append(len(sub_sent) - 1)
                if len(sent_token_ids) % sub_sent_size > 0:
                    sub_sent = sent_token_ids[-sub_sent_size:]
                    x.append(sub_sent[:-1])
                    y.append(sub_sent[1:])
                    seq_lens.append(len(sub_sent) - 1)
            else:
                x.append(sent_token_ids[:-1])
                y.append(sent_token_ids[1:])
                seq_lens.append(len(sent_token_ids) - 1)
    return x, y, seq_lens

In [ ]:
window_size = 40
x_train, y_train, train_seq_lens = get_feature_label(train_token_ids, window_size)
x_valid, y_valid, valid_seq_lens = get_feature_label(valid_token_ids)
x_test, y_test, test_seq_lens = get_feature_label(valid_token_ids)
print(max(train_seq_lens), max(valid_seq_lens), max(test_seq_lens))

40 68 68


### 2.2 Pad sentences in a batch to equal length (5 points)

In [ ]:
def pad_batch(x_batch: List[List[int]], y_batch: List[List[int]], seq_lens_batch: List[int], pad_val: int):
    """Pad the sentences in a batch with pad_val based on the longest sentence.

    Args:
        x_batch, y_batch, seq_lens_batch: The input data (lists of token id lists and their lengths)
        pad_val: The padding value to use.

    Returns:
        x_batch: Tensor of shape (batch_size, max_seq_len)
        y_batch: Tensor of shape (batch_size, max_seq_len)
        seq_lens_batch: Tensor of shape (batch_size,)
    """
    max_len = max(seq_lens_batch)

    padded_x_batch = [seq + [pad_val] * (max_len - len(seq)) for seq in x_batch]
    padded_y_batch = [seq + [pad_val] * (max_len - len(seq)) for seq in y_batch]

    x_tensor = tf.convert_to_tensor(padded_x_batch, dtype=tf.int64)
    y_tensor = tf.convert_to_tensor(padded_y_batch, dtype=tf.int64)
    seq_lens_tensor = tf.convert_to_tensor(seq_lens_batch, dtype=tf.int64)

    return x_tensor, y_tensor, seq_lens_tensor


### 2.3 RNN language model (10 points)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

class RNN(Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        """Init of the RNN model

        Args:
            vocab_size, embedding_dim: Used for initializing the embedding layer.
            hidden_units: Number of hidden units of the RNN layer.
        """
        super().__init__()
        #embedding layer.
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        # simple RNN layer that returns sequences.
        self.rnn = SimpleRNN(units=hidden_units, return_sequences=True)
        #dense layer that maps the RNN output to logits over the vocabulary.
        self.dense = Dense(vocab_size)

    def call(self, x):
        """Forward pass of the RNN model.

        Args:
            x: Tensor, (batch_size x max_seq_len). Input tokens.
               Here, max_seq_len is the longest sentence length in this batch because we did pad_batch.
        Returns:
            outputs: Tensor, (batch_size x max_seq_len x vocab_size). Logits for every time step.

        """

        x_emb = self.embedding(x)

        rnn_ot = self.rnn(x_emb)

        outputs = self.dense(rnn_ot)
        return outputs


### 2.4 Seq2seq loss (5 Points)

In [ ]:
import tensorflow as tf

def seq2seq_loss(logits, target, seq_lens):
    """
    Calculate the sequence-to-sequence loss using sparse categorical crossentropy,
    with a mask to ignore padded timesteps.

    Args:
        logits: Tensor of shape (batch_size, max_seq_len, vocab_size). The output logits from the model.
        target: Tensor of shape (batch_size, max_seq_len). The ground-truth token ids.
        seq_lens: Tensor of shape (batch_size,). The actual sequence lengths (before padding).

    Returns:
        loss: A scalar tensor representing the average sequence loss.
    """
    # Create a mask with 1's for valid timesteps and 0's for padded positions.
    ma = tf.sequence_mask(seq_lens, maxlen=tf.shape(target)[1], dtype=tf.float32)


    loss = tf.keras.losses.sparse_categorical_crossentropy(target, logits, from_logits=True)

    # Apply the mask to zero-out losses from padded positions.
    loss = loss * ma


    loss = tf.reduce_sum(loss) / tf.reduce_sum(ma)

    return loss


In [ ]:
vocab_size = len(tokenizer)
hidden_units = 128
embedding_dim = 64
num_epoch = 30
batch_size = 256

In [ ]:
model = RNN(vocab_size, embedding_dim, hidden_units)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

### 2.5 Train RNN

If you implement everything correctly, the finall loss will be around 5.2

In [ ]:
num_samples = len(x_train)
n_batch = int(np.ceil(num_samples / batch_size))
n_valid_batch = int(np.ceil(len(x_valid) / batch_size))
for epoch in range(num_epoch):
    epoch_loss = 0.0
    for batch_idx in range(n_batch):
        start = batch_idx * batch_size
        end = start + batch_size
        x_batch, y_batch, seq_lens_batch = x_train[start:end], y_train[start:end], train_seq_lens[start:end]
        real_batch_size = len(x_batch)
        x_batch, y_batch, seq_lens_batch = pad_batch(x_batch, y_batch, seq_lens_batch, pad_val=tokenizer.pad_token_id)

        with tf.GradientTape() as tape:
            output = model(x_batch)
            loss = seq2seq_loss(output, y_batch, seq_lens_batch)

        if batch_idx % 1 == 0 or batch_idx == num_samples - 1:
            print_line(f'Epoch {epoch + 1} / {num_epoch} - Step {batch_idx + 1} / {n_batch} - loss: {loss:.4f}')

        trainable_vars = model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        optimizer.apply_gradients(zip(gradients, trainable_vars))
        epoch_loss += loss * real_batch_size

    valid_loss = 0.0
    for batch_idx in range(n_valid_batch):
        start = batch_idx * batch_size
        end = start + batch_size
        x_batch, y_batch, seq_lens_batch = x_valid[start:end], y_valid[start:end], valid_seq_lens[start:end]
        real_batch_size = len(x_batch)
        x_batch, y_batch, seq_lens_batch = pad_batch(x_batch, y_batch, seq_lens_batch, pad_val=tokenizer.pad_token_id)
        output = model(x_batch)
        loss = seq2seq_loss(output, y_batch, seq_lens_batch)

        if batch_idx % 1 == 0 or batch_idx == len(x_valid) - 1:
            print_line(f'Epoch {epoch + 1} / {num_epoch} - Step {batch_idx + 1} / {n_valid_batch} - loss: {loss:.4f}')

        valid_loss += loss * real_batch_size
    print(f'\rEpoch {epoch + 1} / {num_epoch} - Step {n_batch} / {n_batch} - train loss: {epoch_loss / num_samples:.4f} - valid loss: {valid_loss / len(x_valid):.4f}')

Epoch 1 / 30 - Step 170 / 170 - train loss: 6.9330 - valid loss: 6.6278
Epoch 2 / 30 - Step 170 / 170 - train loss: 6.6486 - valid loss: 6.5479
Epoch 3 / 30 - Step 170 / 170 - train loss: 6.4931 - valid loss: 6.3515
Epoch 4 / 30 - Step 170 / 170 - train loss: 6.2797 - valid loss: 6.1636
Epoch 5 / 30 - Step 170 / 170 - train loss: 6.0923 - valid loss: 5.9945
Epoch 6 / 30 - Step 170 / 170 - train loss: 5.9112 - valid loss: 5.8373
Epoch 7 / 30 - Step 170 / 170 - train loss: 5.7523 - valid loss: 5.7204
Epoch 8 / 30 - Step 170 / 170 - train loss: 5.6308 - valid loss: 5.6386
Epoch 9 / 30 - Step 170 / 170 - train loss: 5.5310 - valid loss: 5.5654
Epoch 10 / 30 - Step 170 / 170 - train loss: 5.4435 - valid loss: 5.5079
Epoch 11 / 30 - Step 170 / 170 - train loss: 5.3656 - valid loss: 5.4591
Epoch 12 / 30 - Step 170 / 170 - train loss: 5.2952 - valid loss: 5.4174
Epoch 13 / 30 - Step 170 / 170 - train loss: 5.2323 - valid loss: 5.3821
Epoch 14 / 30 - Step 170 / 170 - train loss: 5.1752 - valid 

### 2.6 Perplexity of RNN (10 points)

Here,
1. you need to calculate the perplexity based on its definition.
2. Besides, you need to record the loss for every word prediction and calculate the sum of loss
3. Finaly, you will need to compare the perplexity by definition and the perplexity by the loss: `np.exp(total_loss / n_words)`

In [ ]:
import math
import numpy as np
import tensorflow as tf

n = len(x_valid)
log_probs = 0
n_words = 0      # Total number of word predictions.
total_loss = 0   # Sum of cross-entropy losses

for i in range(n):
    if i % 1 == 0 or i == n - 1:
        print_line('Calculating perplexity:', (i + 1), '/', n)

    x_line, y_line, line_seq_lens = x_valid[i:i + 1], y_valid[i:i + 1], valid_seq_lens[i:i + 1]
    x_line, y_line, line_seq_lens = pad_batch(x_line, y_line, line_seq_lens, tokenizer.pad_token_id)


    output = model(x_line)
    pred_probs = tf.nn.softmax(output, axis=-1)


    # We assume y_line[0] is a list/tensor of token ids for the sentence.
    for real_token, probs in zip(y_line[0].numpy(), pred_probs[0].numpy()):

        # Convert the real token id to an int and get the probability assigned
        p = probs[int(real_token)]


        log_probs += math.log(p)

        total_loss += -math.log(p)
        n_words += 1

perplexity_by_def = np.exp(-log_probs / n_words)
perplexity_by_loss = np.exp(total_loss / n_words)

print('\n')
print(f'Perplexity by definition: {perplexity_by_def:.4f}, Perplexity by loss: {perplexity_by_loss:.4f}')


Calculating perplexity: 3352 / 3352

Perplexity by definition: 185.2645, Perplexity by loss: 185.2645


### 2.7 Predict the next word given a previous sentence (5 Points)

In [ ]:
def predict_next_token(model, token_ids):

    # Convert token_ids to a tensor with a batch dimension.
    x = tf.convert_to_tensor([token_ids], dtype=tf.int64)

    logits = model(x)
    # Take the logits from the last time step.
    last_logits = logits[0, -1, :]


    pred_token = tf.argmax(last_logits).numpy()
    return pred_token

np.random.seed(12345)
vocab_size = len(tokenizer)
indexes = np.random.choice(len(test_token_ids), 10, replace=False)
for i in indexes:
    token_ids = test_token_ids[i][1:-1]
    print(' '.join([tokenizer.inverse_vocab[token_id] for token_id in token_ids]) + ' ____')
    pred = predict_next_token(model, token_ids)
    print(f'predicted last token: {tokenizer.inverse_vocab[pred]}')
    print('---------------------------------------------')


sharply falling stock prices do reduce consumer wealth damage business ____
predicted last token: </s>
---------------------------------------------
but robert an official of the association said no ____
predicted last token: one
---------------------------------------------
it also has interests in military electronics and marine ____
predicted last token: and
---------------------------------------------
first chicago since n has reduced its loans to such ____
predicted last token: as
---------------------------------------------
david m jones vice president at g ____
predicted last token: s
---------------------------------------------
the n stock specialist firms on the big board floor ____
predicted last token: were
---------------------------------------------
at the same time the business was hurt by ____
predicted last token: the
---------------------------------------------
salomon will cover the warrants by buying sufficient shares or ____
predicted last token: n
------------

## 3. Conclusion (5 points)

Briefly analyze the result of N-Gram and RNN


N-Gram Model:


*   Strengths: Simple, fast, good for common word pairs and short-range dependencies.

* Weaknesses: Struggles with longer-range dependencies, has high perplexity, and lacks flexibility in handling complex or rare sequences.


Perplexity: 325.8354, indicating uncertainty in predictions.

RNN Model:



* Strengths: Captures long-range dependencies, adapts over time with training,and performs better in complex sequence generation.
* Weaknesses: Computationally expensive, slower training, and still struggles with perplexity.  


Perplexity: 185.2645, lower than the N-Gram model, indicating better prediction consistency.


**Conclusion: **
The RNN model outperforms the N-Gram model in handling long-range dependencies and prediction accuracy, though both models still have room for improvement due to their high perplexity. The N-Gram model is faster but kinda limited in its ability to handle complex contexts.